### ***DB Info***

In [3]:
import json

with open('mariadb_dbinfo' , 'r') as f:
    db_info = json.load(f)

### assays

In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import re

base_dir = '/home/ubuntu/toxcast_rawdata/assays'

USERNAME = db_info["username"]
PASSWORD = db_info["password"]
HOST = db_info["host"]
PORT = db_info["port"]
DATABASE = db_info["database"]
ENCODED_PASSWORD = quote_plus(PASSWORD)

engine = create_engine(f"mariadb+mariadbconnector://{USERNAME}:{ENCODED_PASSWORD}@{HOST}:{PORT}/{DATABASE}")

TABLE_NAME = "assays"

for root, dirs, files in os.walk(base_dir):
    for f in files:
        title = result = re.sub(r"-\d{4}-\d{2}-\d{2}\.xlsx$", "", f.replace('Assay List ',''))
        file_path = os.path.join(root, f)
        print(file_path)
        df = pd.read_excel(file_path, sheet_name=0, engine="openpyxl")

        df['ASSAY NAME'] = title

        df = df[["ASSAY NAME"] + [col for col in df.columns if col != "ASSAY NAME"]]

        print(file_path, title)
        print(df.head())

        df.to_sql(TABLE_NAME, con=engine, if_exists="append", index=False)

### chemicals

In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import re

base_dir = '/home/ubuntu/toxcast_rawdata/chemicals'

USERNAME = db_info["username"]
PASSWORD = db_info["password"]
HOST = db_info["host"]
PORT = db_info["port"]
DATABASE = db_info["database"]
ENCODED_PASSWORD = quote_plus(PASSWORD)

engine = create_engine(f"mariadb+mariadbconnector://{USERNAME}:{ENCODED_PASSWORD}@{HOST}:{PORT}/{DATABASE}")

TABLE_NAME = "chemicals"

for root, dirs, files in os.walk(base_dir):
    for f in files:
        file_path = os.path.join(root, f)
        try:
            print(file_path)

            title = re.sub(r"-\d{4}-\d{2}-\d{2}\.xlsx$", "", f.replace('Chemical List ',''))
            
            df = pd.read_excel(file_path, sheet_name=0, engine="openpyxl")

            df['CHEMICAL NAME'] = title

            df = df[["CHEMICAL NAME"] + [col for col in df.columns if col != "CHEMICAL NAME"]]

            print(title)
            print(df.head())

            df.to_sql(TABLE_NAME, con=engine, if_exists="append", index=False)
        except:
            print('pass', file_path)

### bioactivities

In [ ]:
import pandas as pd
import re
import os
import numpy as np
import pymysql

base_dir = '/home/ubuntu/toxcast_rawdata/bioactivity'
bioactivity_dtxsid_dict = {}
with open('/home/ubuntu/toxcast_rawdata/bioactivity_dtxsid.txt', 'r') as f:
    lines = f.readlines()
    for l in lines:
        bioactivity_dtxsid_pair = l.strip().split('|')
        bioactivity_dtxsid_dict[bioactivity_dtxsid_pair[0].replace('%','_').replace("'","_").replace("/","_").replace(":","_").replace(":","_")] = bioactivity_dtxsid_pair[1]

connection = pymysql.connect(
    host=db_info["host"],
    port=db_info["port"],
    database=db_info["database"],
    user=db_info["username"],
    password=db_info["password"])
cursor = connection.cursor()

double_columns = ['CONTINUOUS_HIT_CALL', 'TOP', 'SCALED_TOP', 'AC50', 'LOGAC50', 'CUTOFF', 'ACC', 'AC10', 'BMR', 'BMD', 'BMAD', 'MAX_MED_CONC', 'STOCK_CONC', 'CYTOTOX_BURST']

i = 0
for root, dirs, files in os.walk(base_dir):
    for f in files:
        i += 1
        
        file_path = os.path.join(root, f)
        print(i, file_path)
        try:    
            title = re.sub(r' Toxcast Summary -\d{4}-\d{2}-\d{2}\.xlsx| Toxcast Summary -\d{4}-\d{2}-\d{2}\.zip', "", f)

            dtxsid = bioactivity_dtxsid_dict[title]
            
            df = pd.read_excel(file_path, sheet_name=0, engine="openpyxl", dtype=object)
            df = df.where(pd.notnull(df), None)

            for double_column in double_columns:
                df[double_column] = df[double_column].replace(" -  ", np.nan)

            df['TITLE'] = title
            df['DTXSID'] = dtxsid
            ignored_column = ['TITLE', 'DTXSID']

            df = df[["TITLE"] + ['DTXSID'] + [col for col in df.columns if col not in ignored_column]]

            df = df.where(pd.notnull(df), None)

            df.replace({np.nan: None}, inplace=True)

            #print(df)

            
            data = df.values.tolist()

            # Prepare your query
            query = "INSERT INTO bioactivity (TITLE, DTXSID, NAME, ASSAY_ENDPOINT_DESCRIPTION, ASSAY_COMPONENT_NAME, ASSAY_COMPONENT_DESCRIPTION, DESCRIPTION, SEQAPASS, GENE_SYMBOL, GENE_URL, GENE_NAME, AOP, EVENT, HIT_CALL, CONTINUOUS_HIT_CALL, TOP, SCALED_TOP, AC50, LOGAC50, CUTOFF, ACC, AC10, BMR, BMD, BMAD, MAX_MED_CONC, NORMALIZED_DATA_TYPE, ASSAY_SOURCE, ASSAY_NAME, ASSAY_DESC, ASSAY_FUNCTION_TYPE, FLAGS, STOCK_CONC, STOCK_CONC_UNITS, ORGANISM, CELL_LINE, CELL_FORMAT, CONCENTRATION, RESPONSE, CYTOTOX_BURST, INTENDED_TARGET_FAMILY, INTENDED_TARGET_TYPE, INTENDED_TARGET_TYPE_SUB, INTENDED_TARGET_FAMILY_SUB, BIOLOGICAL_PROCESS_TARGET, DETECTION_TECHNOLOGY_TYPE, TISSUE, SAMPLE_ID, REPR) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"


            cursor.executemany(query, data)
            connection.commit()
        except:
            pass

In [ ]:
import pandas as pd
import os
import re

base_dir = '/home/ubuntu/toxcast_rawdata/bioactivity'

collected_bioactivities = []
bioactivity_dtxsid_dict = {}
collected_dtxsid = []
unique_bioactivities = {}

dtxsid_df = pd.read_csv('/home/ubuntu/toxcast_rawdata/bioactivity_dtxsid.txt', sep='|', header=None)
dtxsid_df.columns = ['TITLE', 'DTXSID']

bioactivity_dtxsid_dict = {}
with open('/home/ubuntu/toxcast_rawdata/bioactivity_dtxsid.txt', 'r') as f:
    lines = f.readlines()
    for l in lines:
        bioactivity_dtxsid_pair = l.strip().split('|')
        bioactivity_dtxsid_dict[bioactivity_dtxsid_pair[0].replace('%','_').replace("'","_").replace("/","_").replace(":","_").replace(":","_")] = bioactivity_dtxsid_pair[1]

with open('/home/ubuntu/toxcast_rawdata/uniq_bioactivities.txt', 'r') as f:
    lines = f.readlines()
    for l in lines:
        unique_bioactivities[l.strip()] = True

i = 0
for root, dirs, files in os.walk(base_dir):
    for f in files:
        file_path = os.path.join(root, f)
        print(i, file_path)

        i += 1

        title = re.sub(r' Toxcast Summary -\d{4}-\d{2}-\d{2}\.xlsx| Toxcast Summary -\d{4}-\d{2}-\d{2}\.zip', "", f)

        dtxsid = bioactivity_dtxsid_dict[title]
        
        # dtxsid = dtxsid_df.where( dtxsid_df['TITLE'] == title).dropna()['DTXSID'].iloc[0]


        unique_bioactivities.pop(dtxsid)


print(unique_bioactivities)

